In [22]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from database_models import Base, MemRam, Arena, Pool, Block, Ledger, StoredObject
from sqlalchemy.sql import func
import helpers.listeners
import json
import re
import hashlib



In [23]:
class MemManager:
    def __init__(self, db_url: str) -> None:

        self.engine = create_engine(db_url)
        Base.metadata.create_all(self.engine)
        Session = sessionmaker(bind=self.engine)
        self.session = Session()

        # Skapa en ny MemRam-post
        self.memram = MemRam()
        self.session.add(self.memram)
        self.session.commit()

    def add_arena(self) -> Arena:
        """Create a new arena and add it to the MemRam table"""
        new_arena = Arena()
        new_arena.memram = self.memram
        self.session.add(new_arena)
        self.session.commit()
        return new_arena

    def add_pool(self, arena: Arena) -> Pool:
        """Create a new pool and add it to the arena table"""
        new_pool = Pool()
        new_pool.arena = arena
        self.session.add(new_pool)
        self.session.commit()
        return new_pool

    def add_block(self, pool: Pool) -> Block:
        """Create a new block and add it to the pool table"""
        new_block = Block()
        new_block.pool = pool
        self.session.add(new_block)
        self.session.commit()
        return new_block

    def generate_object_id(self, identifier):
        """Generate a consistent object_id for a given identifier."""
        if isinstance(identifier, str):
            # Check if the string is already a SHA-256 hash
            if re.fullmatch(r'[a-f0-9]{64}', identifier):
                return identifier
            else:
                # Assume the string is an object and hash it
                serialized_obj = json.dumps(identifier, sort_keys=True)
                return hashlib.sha256(serialized_obj.encode('utf-8')).hexdigest()
        else:
            # Serialize and hash the object
            serialized_obj = json.dumps(identifier, sort_keys=True)
            return hashlib.sha256(serialized_obj.encode('utf-8')).hexdigest()

    def allocate_memory_for_object(self, obj_instance) -> None:
        """Allocate memory for an object by creating necessary arenas, pools and blocks."""
        obj_size = obj_instance.__sizeof__()

        # Create a unique and consistent identifier for the object
        object_id = self.generate_object_id(obj_instance)

        if self.memram.max_mem < obj_size:
            raise MemoryError("Not enough memory to allocate object.")

        remaining_size = obj_size
        blocks_to_update = []

        # Check if the object is already stored
        stored_object = self.session.query(StoredObject).filter(StoredObject.object_id == object_id).first()
        if stored_object:
            print(f"Object with identifier {object_id} already exists in the database.")
            return

        # Store the object in the StoredObject table
        stored_object = StoredObject(object_id=object_id, object_data=obj_instance)
        self.session.add(stored_object)
        self.session.commit()

        # Get blocks that have enough space for the object
        while remaining_size > 0:
            suitable_block = self.session.query(Block).filter(
                Block.is_free == 1,
                Block.max_mem - Block.mem > 0
            ).first()

            if suitable_block:
                # Calculate how much space is left in the block
                available_space = suitable_block.max_mem - suitable_block.mem
                to_allocate = min(remaining_size, available_space)

                # Add part of the object to the block
                suitable_block.mem += to_allocate
                suitable_block.is_free = 0 if suitable_block.mem == suitable_block.max_mem else 1
                remaining_size -= to_allocate
                blocks_to_update.append(suitable_block)

                # Add a post to the ledger
                ledger_entry = Ledger(
                    arena_id=suitable_block.pool.arena_id,
                    pool_id=suitable_block.pool_id,
                    block_id=suitable_block.id,
                    object_id=object_id,
                    allocated_mem=to_allocate
                )
                self.session.add(ledger_entry)
            else:
                # Find an arena with enough space for a new pool
                arena = self.session.query(Arena).filter(Arena.max_mem - Arena.mem > 0).first()
                if not arena:
                    arena = self.add_arena()

                # Check if there are enough pools in the arena
                pool = self.session.query(Pool).filter(Pool.arena_id == arena.id, Pool.max_mem - Pool.mem > 0).first()
                if not pool:
                    pool = self.add_pool(arena)

                # Create a new block in the pool
                block = self.add_block(pool)

                # Calculate how much space is left in the block
                available_space = block.max_mem - block.mem
                to_allocate = min(remaining_size, available_space)

                # Add part of the object to the block
                block.mem += to_allocate
                block.is_free = 0 if block.mem == block.max_mem else 1
                remaining_size -= to_allocate
                blocks_to_update.append(block)

                # Add a post to the ledger
                ledger_entry = Ledger(
                    arena_id=block.pool.arena_id,
                    pool_id=block.pool_id,
                    block_id=block.id,
                    object_id=object_id,
                    allocated_mem=to_allocate
                )
                self.session.add(ledger_entry)

        # Batch commit
        self.session.bulk_save_objects(blocks_to_update)
        self.session.commit()

        print(f"Allocated {obj_size} bytes for object across multiple blocks.")

    def free_memory_for_object(self, identifier) -> None:
        """Free memory for an object by updating the ledger and blocks."""
        # Check if the identifier is an object or a string
        object_id = self.generate_object_id(identifier)

        print(f"Freeing memory for object with identifier: {object_id}")

        # Get all ledger entries for the given object_id
        ledger_entries = self.session.query(Ledger).filter(Ledger.object_id == object_id).all()

        for ledger_entry in ledger_entries:
            # Get the corresponding block
            block = self.session.query(Block).filter(Block.id == ledger_entry.block_id).first()

            if block:
                # Free memory in the block
                allocated_mem = ledger_entry.allocated_mem
                block.mem -= allocated_mem
                # Mark the block as dirty to trigger the listener
                block.is_free = 0 if block.mem == block.max_mem else 1

        # Delete all ledger entries for the given object_id
        self.session.query(Ledger).filter(Ledger.object_id == object_id).delete()
        self.session.query(StoredObject).filter(StoredObject.object_id == object_id).delete()

        # Save the changes
        self.session.commit()

    def get_object(self, identifier):
        """Retrieve an object from the database using its identifier."""
        # Generate the object_id from the identifier
        object_id = self.generate_object_id(identifier)

        # Query the StoredObject table for the object
        stored_object = self.session.query(StoredObject).filter(StoredObject.object_id == object_id).first()

        if stored_object:
            print(f"Object {object_id} retrieved from the database.")
            return stored_object.object_data
        else:
            print(f"Object {object_id} not found in the database.")
            return None

In [24]:
mem_manager = MemManager(r'sqlite:///database.db')

In [25]:
# Allokera minne för ett objekt
obj = "Hello, World!"  # Exempelobjekt
mem_manager.allocate_memory_for_object(obj)

Allocated 62 bytes for object across multiple blocks.


In [26]:
obj = 123*2
mem_manager.allocate_memory_for_object(obj)

Allocated 28 bytes for object across multiple blocks.


In [20]:
mem_manager.free_memory_for_object("cc82ebbcf8b60a5821d1c51c72cd79380ecea47de343ccb3b158938a2b3bf764")

Freeing memory for object with identifier: cc82ebbcf8b60a5821d1c51c72cd79380ecea47de343ccb3b158938a2b3bf764


In [7]:
mem_manager.get_object("37c20f19f3272b5ccc3a5d80587eb9deb3f4afcf568c4280fb195568da8eb1a2")

Object 37c20f19f3272b5ccc3a5d80587eb9deb3f4afcf568c4280fb195568da8eb1a2 retrieved from the database.


246

In [21]:
mem_manager.free_memory_for_object("37c20f19f3272b5ccc3a5d80587eb9deb3f4afcf568c4280fb195568da8eb1a2")

Freeing memory for object with identifier: 37c20f19f3272b5ccc3a5d80587eb9deb3f4afcf568c4280fb195568da8eb1a2


In [8]:
mem_manager.free_memory_for_object(obj)

Freeing memory for object with identifier: c6f9df8b40255b636b9b38f35dfecad50bdaf7797fa1f8a608c65ac2c4ce1cca
Listener: Updating Block ID 1: is_free set to 1
Listener: Updating Pool ID 1: mem set to 3646
Listener: Updating Arena ID 1: mem set to 5461
Listener: Updating MemRam ID 1: mem set to 5461
Listener: Updating Block ID 2: is_free set to 1
Listener: Updating Pool ID 1: mem set to 3134
Listener: Updating Arena ID 1: mem set to 4949
Listener: Updating MemRam ID 1: mem set to 4949
Listener: Updating Block ID 3: is_free set to 1
Listener: Updating Pool ID 1: mem set to 2622
Listener: Updating Arena ID 1: mem set to 4437
Listener: Updating MemRam ID 1: mem set to 4437
Listener: Updating Block ID 4: is_free set to 1
Listener: Updating Pool ID 1: mem set to 2110
Listener: Updating Arena ID 1: mem set to 3925
Listener: Updating MemRam ID 1: mem set to 3925
Listener: Updating Block ID 5: is_free set to 1
Listener: Updating Pool ID 1: mem set to 1598
Listener: Updating Arena ID 1: mem set to 

In [16]:
obj = "something very long something diffeernt then anything else" *100
mem_manager.allocate_memory_for_object(obj)

Allocated 5849 bytes for object across multiple blocks.


In [17]:
mem_manager.free_memory_for_object(obj)

Freeing memory for object with identifier: c6f9df8b40255b636b9b38f35dfecad50bdaf7797fa1f8a608c65ac2c4ce1cca
